<a href="https://colab.research.google.com/github/jen0525/AIFFEL_quest_cr/blob/master/data_collection/wikipedia_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Shakespeare 관련 정보 크롤링 (Wikipedia)

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from google.colab import drive
import os

# Step 1: 수집할 웹 페이지 URL 설정 (Shakespeare 관련 정보가 있는 위키피디아 페이지)
base_url = 'https://en.wikipedia.org'
start_url = 'https://en.wikipedia.org/wiki/William_Shakespeare'

# Step 2: 웹 페이지 요청 및 HTML 파싱
response = requests.get(start_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 3: 페이지에서 내부 링크 수집 (Shakespeare 관련 추가 정보 링크)
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.startswith('/wiki/') and not ':' in href and 'Main_Page' not in href:
        full_url = base_url + href
        if full_url not in links:
            links.append(full_url)

# Step 4: 각 링크에서 데이터 크롤링 함수 정의
def scrape_data_from_link(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 페이지 제목 추출
        title = soup.title.get_text() if soup.title else 'No Title'

        # 본문 내용 추출 (예: <p> 태그 안의 내용)
        paragraphs = soup.find_all('p')
        content = "\n".join([p.get_text() for p in paragraphs])

        return {'url': url, 'title': title, 'content': content}
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return {'url': url, 'title': 'Error', 'content': 'Error occurred'}

# Step 5: Google Drive 마운트하여 파일 저장
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/shakespeare_data'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# Step 6: 모든 링크에서 데이터 크롤링 및 Chunking으로 저장
chunk_size = 100  # 파일당 100개의 데이터로 나눠 저장
chunk_number = 1

for i in range(0, len(links), chunk_size):
    chunk_links = links[i:i + chunk_size]
    scraped_data = [scrape_data_from_link(link) for link in chunk_links]

    # Step 7: 크롤링된 데이터를 Google Drive에 CSV로 저장 (Chunk별로 저장)
    csv_file_name = f'{drive_path}/scraped_shakespeare_data_part_{chunk_number}.csv'
    with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['URL', 'Title', 'Content'])  # 헤더
        for data in scraped_data:
            writer.writerow([data['url'], data['title'], data['content']])

    chunk_number += 1

print("All chunks saved to Google Drive.")